Goal of this notebook:

Investigate the impact of using discretized datasets (log2, log10, and mdlp) on the classification accuracy with decision tree, random forest and XGBoost.

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
import warnings
import classifier_utils
import configs
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

# Decision Tree

In [4]:
dt = DecisionTreeClassifier(random_state=99)

In [5]:
result_dt_default = classifier_utils.ProjectsResults(dt, projects, non_features_columns)

In [6]:
report_dt_default = result_dt_default.get_report_df(include_overall=True)
# report_dt_default

In [7]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log10 = result_dt_log10.get_report_df(include_overall=True)
# report_dt_log10

In [8]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log2 = result_dt_log2.get_report_df(include_overall=True)
# report_dt_log2

In [9]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_mdlp = result_dt_mdlp.get_report_df(include_overall=True)

In [10]:
df_inner_dt = pd.merge(report_dt_default, report_dt_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.9720,0.96700,0.9720,0.96900,-0.005144,0.000000,-0.003086
1,apache__directory-server,0.9260,0.92000,0.9230,0.88500,-0.006479,-0.003240,-0.044276
2,jgralab__jgralab,0.8340,0.75600,0.8330,0.79500,-0.093525,-0.001199,-0.046763
3,CloudStack-extras__CloudStack-archive,0.7540,0.75500,0.7560,0.73900,0.004065,0.008130,-0.019894
4,Unidata__thredds,0.9010,0.89900,0.9030,0.84900,-0.002220,0.020202,-0.057714
5,apache__accumulo,0.8380,0.83100,0.8300,0.82900,-0.008353,-0.009547,-0.010740
6,Ramblurr__Anki-Android,0.7020,0.64200,0.6590,0.68200,-0.085470,-0.061254,-0.028490
7,getrailo__railo,0.6540,0.63100,0.6450,0.64500,-0.035168,-0.013761,-0.013761
8,TeamDev-Ltd__OpenFaces,0.9650,0.95800,0.9650,0.96500,-0.007254,0.000000,0.000000
9,apache__lucene-solr,0.5630,0.57700,0.5750,0.53300,0.032037,0.027460,-0.053286


## Including NA values

In [11]:
import importlib
importlib.reload(classifier_utils)
result_dt_default_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns, replace_na=True)

In [12]:
report_dt_default_wna = result_dt_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [13]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log2_wna = result_dt_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [14]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log10_wna = result_dt_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [15]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_mdlp_wna = result_dt_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [16]:
df_inner_dt = pd.merge(report_dt_default_wna, report_dt_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97000,0.97100,0.9710,0.9690,0.033333,0.033333,-0.001031
1,apache__directory-server,0.90200,0.89000,0.8770,0.8850,-0.013304,-0.027716,-0.018847
2,jgralab__jgralab,0.82300,0.75100,0.8180,0.7950,-0.087485,-0.006075,-0.034022
3,CloudStack-extras__CloudStack-archive,0.78000,0.76300,0.7720,0.7470,-0.021795,-0.010256,-0.042308
4,Unidata__thredds,0.89800,0.89100,0.8930,0.8520,-0.007795,-0.005568,-0.051225
5,apache__accumulo,0.85500,0.84700,0.8490,0.8480,-0.009357,-0.007018,-0.008187
6,getrailo__railo,0.72900,0.71500,0.7240,0.7090,-0.019204,-0.006859,-0.027435
7,Ramblurr__Anki-Android,0.68000,0.67500,0.6660,0.6880,-0.007353,-0.020588,0.025000
8,TeamDev-Ltd__OpenFaces,0.95400,0.95800,0.9530,0.9550,0.086957,-0.001048,0.021739
9,freenet__fred,0.62600,0.62100,0.6060,0.6200,-0.007987,-0.031949,-0.009585


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# Random forest

In [17]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=400, max_features=0.3, min_samples_leaf=1)# Decision Tree

In [18]:
result_rf_default = classifier_utils.ProjectsResults(rf, projects, non_features_columns)

In [19]:
report_rf_default = result_rf_default.get_report_df(include_overall=True)
# report_rf_default

In [20]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log10 = result_rf_log10.get_report_df(include_overall=True)
# report_rf_log10

In [21]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log2 = result_rf_log2.get_report_df(include_overall=True)
# report_dt_log2

In [22]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_mdlp = result_rf_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [23]:
df_inner_rf = pd.merge(report_rf_default, report_rf_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97600,0.9740,0.9750,0.97300,-0.002049,-0.001025,-0.003074
1,apache__directory-server,0.94200,0.9230,0.9330,0.91400,-0.020170,-0.009554,-0.029724
2,jgralab__jgralab,0.87000,0.8140,0.8670,0.83300,-0.064368,-0.003448,-0.042529
3,CloudStack-extras__CloudStack-archive,0.80400,0.8030,0.8050,0.77100,-0.001244,0.005102,-0.041045
4,Unidata__thredds,0.92100,0.9260,0.9210,0.86200,0.063291,0.000000,-0.064061
5,apache__accumulo,0.86300,0.8650,0.8660,0.85500,0.014599,0.021898,-0.009270
6,Ramblurr__Anki-Android,0.73600,0.7340,0.7420,0.71500,-0.002717,0.022727,-0.028533
7,apache__lucene-solr,0.65000,0.6420,0.6520,0.58600,-0.012308,0.005714,-0.098462
8,getrailo__railo,0.70300,0.6920,0.7110,0.70300,-0.015647,0.026936,0.000000
9,TeamDev-Ltd__OpenFaces,0.97000,0.9680,0.9690,0.97100,-0.002062,-0.001031,0.033333


## Including NA values

In [24]:
import importlib
importlib.reload(classifier_utils)
result_rf_default_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns, replace_na=True)

In [25]:
report_rf_default_wna = result_rf_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [26]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log2_wna = result_rf_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [27]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log10_wna = result_rf_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [28]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_mdlp_wna = result_rf_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [29]:
df_inner_rf = pd.merge(report_rf_default_wna, report_rf_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97700,0.9750,0.9760,0.97500,-0.002047,-0.001024,-0.002047
1,apache__directory-server,0.93300,0.9240,0.9270,0.91400,-0.009646,-0.006431,-0.020364
2,jgralab__jgralab,0.86500,0.8160,0.8610,0.83300,-0.056647,-0.004624,-0.036994
3,CloudStack-extras__CloudStack-archive,0.82200,0.8240,0.8200,0.78600,0.011236,-0.002433,-0.043796
4,Unidata__thredds,0.91500,0.9230,0.9180,0.87800,0.094118,0.035294,-0.040437
5,apache__accumulo,0.88000,0.8750,0.8800,0.87200,-0.005682,0.000000,-0.009091
6,Ramblurr__Anki-Android,0.76200,0.7480,0.7480,0.72600,-0.018373,-0.018373,-0.047244
7,getrailo__railo,0.76900,0.7660,0.7770,0.76600,-0.003901,0.034632,-0.003901
8,apache__lucene-solr,0.65400,0.6230,0.6480,0.58400,-0.047401,-0.009174,-0.107034
9,TeamDev-Ltd__OpenFaces,0.96100,0.9610,0.9590,0.96100,0.000000,-0.002081,0.000000


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# XGBoost Random Forest

In [30]:
xg = XGBRFClassifier(random_state=99, subsample=0.9, eval_metric='mlogloss', n_estimators=400, colsample_bynode=0.4)

In [31]:
result_xg_default = classifier_utils.ProjectsResults(xg, projects, non_features_columns, drop_na=False)

In [32]:
report_xg_default = result_xg_default.get_report_df(include_overall=True)
# report_rf_default

In [33]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_xg_log10 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log10 = result_xg_log10.get_report_df(include_overall=True)
# report_rf_log10

In [34]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_xg_log2 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log2 = result_xg_log2.get_report_df(include_overall=True)
# report_dt_log2

In [35]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_xg_mdlp = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_mdlp = result_xg_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [36]:
df_inner_xg = pd.merge(report_xg_default, report_xg_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_xg_log2 = report_xg_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_log2, on='project', how='inner')
df_inner_xg_mdlp = report_xg_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_mdlp, on='project', how='inner')

accuracy_xg = df_inner_xg.filter(regex=("project|accuracy.*")).copy()
accuracy_xg['improv._log10'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_xg['improv._log2'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_xg['improv._mdlp'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_xg

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97400,0.97200,0.9720,0.9740,-0.002053,-0.002053,0.000000
1,apache__directory-server,0.91800,0.91000,0.9080,0.8980,-0.008715,-0.010893,-0.021786
2,jgralab__jgralab,0.81900,0.71800,0.7980,0.7840,-0.123321,-0.025641,-0.042735
3,Unidata__thredds,0.90500,0.90100,0.9020,0.8470,-0.004420,-0.003315,-0.064088
4,getrailo__railo,0.76400,0.74600,0.7640,0.7340,-0.023560,0.000000,-0.039267
5,apache__accumulo,0.85600,0.83900,0.8510,0.8520,-0.019860,-0.005841,-0.004673
6,Ramblurr__Anki-Android,0.72000,0.71100,0.7090,0.7110,-0.012500,-0.015278,-0.012500
7,TeamDev-Ltd__OpenFaces,0.96100,0.95700,0.9600,0.9590,-0.004162,-0.001041,-0.002081
8,CloudStack-extras__CloudStack-archive,0.71100,0.70600,0.6970,0.6780,-0.007032,-0.019691,-0.046414
9,apache__lucene-solr,0.59100,0.58300,0.5960,0.5790,-0.013536,0.012225,-0.020305


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.